# TODO
- [Instructions](https://opensearch-project.github.io/opensearch-py-ml/examples/demo_deploy_cliptextmodel.html) on documentation missing 
- Tried many ways but I believe mlcommons framework doesn't support CLIP 
- As per [this](https://opensearch.org/docs/latest/ml-commons-plugin/custom-local-models/) only local text, local sparse, cross-encoder and question-answering models are allowed for local models

In [6]:
from transformers import CLIPProcessor, CLIPTextModel
import torch

import opensearch_py_ml as oml
from opensearch_py_ml.ml_commons import MLCommonClient
from opensearchpy import OpenSearch

import warnings
warnings.filterwarnings("ignore", message="Unverified HTTPS request")
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings("ignore", message="TracerWarning: torch.tensor")
warnings.filterwarnings("ignore", message="using SSL with verify_certs=False is insecure.")

In [7]:
# CLUSTER_URL = 'https://192.18.0.111:9200'
CLUSTER_URL = {'host': '192.168.0.111', 'port': 9200}

def get_os_client(cluster_url = CLUSTER_URL,
                  username='admin',
                  password='Developer@123'):
    '''
    Get OpenSearch client
    :param cluster_url: cluster URL like https://ml-te-netwo-1s12ba42br23v-ff1736fa7db98ff2.elb.us-west-2.amazonaws.com:443
    :return: OpenSearch client
    '''
    client = OpenSearch(
        hosts=[cluster_url], #[cluster_url], # {'host': '192.168.0.111', 'port': 9200}
        http_auth=(username, password),
        verify_certs=False,
        ssl_assert_hostname = False,
        ssl_show_warn = False,
        use_ssl=True
    )
    return client

client = get_os_client()

# Update cluster settings to allow local model registration
client.cluster.put_settings(body={
    "persistent": {
        "plugins": {
            "ml_commons": {
                "allow_registering_model_via_url": "true",
                "allow_registering_model_via_local_file": "true",
                "only_run_on_ml_node": "false",
                "model_access_control_enabled": "true",
                "native_memory_threshold": "99"
            }
        }
    }
})

# Connect to ml_common client with OpenSearch client
ml_client = MLCommonClient(client)

In [4]:
from transformers import AutoTokenizer
model_name = "openai/clip-vit-base-patch32" #See https://huggingface.co/models for other options
text_to_encode = "example search query" #See https://huggingface.co/docs/transformers/torchscript for more info on dummy inputs

# Instantiate CLIPTextModel and CLIPProcessor with pretrained weights
model = CLIPTextModel.from_pretrained(model_name, torchscript=True, return_dict=False)
tokenizer = AutoTokenizer.from_pretrained(model_name)
processor = CLIPProcessor.from_pretrained(model_name)

# Use processor to generate tensors and create dummy input
text_inputs = processor(text=text_to_encode, return_tensors="pt",max_length=77, padding="max_length", truncation=True)
dummy_input = [text_inputs['input_ids'], text_inputs['attention_mask']]

# Trace model and convert to torchscript object
traced_model = torch.jit.trace(model, dummy_input)

# Save model in portable format
torch.jit.save(traced_model, "clip-vit-base-patch32.pt")

# Save the tokenizer
tokenizer.save_pretrained("tokenizer")

# Save the model configuration
model.config.save_pretrained("model_config")

/home/ubuntu/git-projects/personal/github.com/elasticsearch_opensearch/.venv/lib/python3.8/site-packages/transformers/modeling_attn_mask_utils.py:86: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if input_shape[-1] > 1 or self.sliding_window is not None:
/home/ubuntu/git-projects/personal/github.com/elasticsearch_opensearch/.venv/lib/python3.8/site-packages/transformers/modeling_attn_mask_utils.py:162: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if past_key_values_length > 0:


In [35]:
import json
# read model configuration from model_config/config.json file
with open('model_config/config.json') as f:
    all_config = json.load(f)

mlcommons_model_config = {}
mlcommons_model_config['name'] = "clip-vit-base-patch32"
mlcommons_model_config['version'] = '1.0.0'
mlcommons_model_config['model_format'] = 'TORCH_SCRIPT'

model_config = {}
model_config['embedding_dimension'] = 512
model_config['model_type'] = 'clip'
model_config['framework_type'] = 'huggingface_transformers'
model_config['pooling_mode'] = 'MEAN'
model_config['normalize_result'] = 'true'
# model_config['all_config'] = json.dumps(all_config)

mlcommons_model_config['model_config'] = model_config

# save mlcommons model configuration to mlcommons_model_config.json file
with open('mlcommons_model_config.json', 'w') as f:
    json.dump(mlcommons_model_config, f)

print(mlcommons_model_config)


{'name': 'clip-vit-base-patch32', 'version': '1.0.0', 'model_format': 'TORCH_SCRIPT', 'model_config': {'embedding_dimension': 512, 'model_type': 'clip', 'framework_type': 'huggingface_transformers', 'pooling_mode': 'MEAN', 'normalize_result': 'true'}}


In [36]:
model_path = "clip-vit-base-patch32.zip"
model_config_path = "mlcommons_model_config.json"

model_id_file_system = ml_client.register_model(model_path, model_config_path, isVerbose=True, deploy_model = True)


Total number of chunks 19
Sha1 value of the model file:  14f97536887d5ced13e15564531a07a8176f528db4ece1984af5bade720ca240
Model meta data was created successfully. Model Id:  VpoCWZIBg2jaWXNprLTn
uploading chunk 1 of 19
Model id: {'status': 'Uploaded'}
uploading chunk 2 of 19
Model id: {'status': 'Uploaded'}
uploading chunk 3 of 19
Model id: {'status': 'Uploaded'}
uploading chunk 4 of 19
Model id: {'status': 'Uploaded'}
uploading chunk 5 of 19
Model id: {'status': 'Uploaded'}
uploading chunk 6 of 19
Model id: {'status': 'Uploaded'}
uploading chunk 7 of 19
Model id: {'status': 'Uploaded'}
uploading chunk 8 of 19
Model id: {'status': 'Uploaded'}
uploading chunk 9 of 19
Model id: {'status': 'Uploaded'}
uploading chunk 10 of 19
Model id: {'status': 'Uploaded'}
uploading chunk 11 of 19
Model id: {'status': 'Uploaded'}
uploading chunk 12 of 19
Model id: {'status': 'Uploaded'}
uploading chunk 13 of 19
Model id: {'status': 'Uploaded'}
uploading chunk 14 of 19
Model id: {'status': 'Uploaded'}
u

KeyboardInterrupt: 

In [ ]:
import base64
from PIL import Image
import io

def image_to_base64(image_path):
    # Open the image
    with Image.open(image_path) as img:
        # Convert image to RGB if it's not
        if img.mode != 'RGB':
            img = img.convert('RGB')
        
        # Create a byte stream
        buffered = io.BytesIO()
        # Save the image to the byte stream in JPEG format
        img.save(buffered, format="JPEG")
        # Encode the byte stream to base64
        img_str = base64.b64encode(buffered.getvalue()).decode('utf-8')
    
    return img_str

# Now you can use image_binary in your OpenSearch document
document1 = {
    "image_description": "bear holding toilet paper",
    "image_binary": image_to_base64("test.png")
}

document2 = {
    "image_description": "different diaper brands",
    "image_binary": image_to_base64("diapers.png")
}

# Index the document in OpenSearch
client.index(index="my-nlp-index-1", body=document1)
client.index(index="my-nlp-index-1", body=document2)

In [ ]:
# TODO